In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

VER = 172
USE = 'orders'

In [2]:
import pandas as pd, cudf, numpy as np, gc
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt

In [3]:
LOAD = 709
OFFSET = ''

df = pd.read_csv(f"../../data/candidate_scores/submission_v{LOAD}{OFFSET}.csv")
df = df.loc[df.session_type.str.contains(USE)]
df = cudf.DataFrame(df)
print( df.shape )
df.head()

(1801251, 2)


,session_type,labels
1801251,11098528_orders,11830 1732105 588923 1157882 884502 1182614 87...
1801252,11098529_orders,1105029 1049489 53004 1694360 628708 459126 13...
1801253,11098530_orders,409236 264500 1603001 963957 254154 364155 583...
1801254,11098531_orders,396199 1271998 452188 1728212 1365569 624163 1...
1801255,11098532_orders,876469 7651 1159379 476681 669555 1202618 1081...


In [4]:
OFFSET = '_wgt'

df2 = pd.read_csv(f"../../data/candidate_scores/submission_v{LOAD}{OFFSET}.csv")
df2 = df2.loc[df2.session_type.str.contains(USE)]
df2 = cudf.DataFrame(df2)
print( df2.shape )
df2.head()

(1801251, 2)


,session_type,labels
1801251,11098528_orders,-1 36.65 35.15 29.149999999999995 28.950000000...
1801252,11098529_orders,-1 19.275 13.125 12.174999999999999 10.55 10.3...
1801253,11098530_orders,-1 -1 48.825 33.375 33.025 30.175 28.325000000...
1801254,11098531_orders,-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 22.0272727272...
1801255,11098532_orders,-1 -1 25.575 25.275000000000002 23.50000000000...


In [5]:
df['user'] = df.session_type.str.partition('_')[0].astype('int32')
df = df.drop('session_type',axis=1)
df.head()

,labels,user
1801251,11830 1732105 588923 1157882 884502 1182614 87...,11098528
1801252,1105029 1049489 53004 1694360 628708 459126 13...,11098529
1801253,409236 264500 1603001 963957 254154 364155 583...,11098530
1801254,396199 1271998 452188 1728212 1365569 624163 1...,11098531
1801255,876469 7651 1159379 476681 669555 1202618 1081...,11098532


In [6]:
df2['user'] = df2.session_type.str.partition('_')[0].astype('int32')
df2 = df2.drop('session_type',axis=1)
df2.head()

,labels,user
1801251,-1 36.65 35.15 29.149999999999995 28.950000000...,11098528
1801252,-1 19.275 13.125 12.174999999999999 10.55 10.3...,11098529
1801253,-1 -1 48.825 33.375 33.025 30.175 28.325000000...,11098530
1801254,-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 22.0272727272...,11098531
1801255,-1 -1 25.575 25.275000000000002 23.50000000000...,11098532


In [7]:
tmp = df.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
tmp['pos'] = np.tile( np.arange(100),tmp.shape[0]//100 ).astype('int32')
tmp.head()

,item,pos
1801251,11830,0
1801251,1732105,1
1801251,588923,2
1801251,1157882,3
1801251,884502,4


In [8]:
users = df[['user']].copy()
users.head()

,user
1801251,11098528
1801252,11098529
1801253,11098530
1801254,11098531
1801255,11098532


In [9]:
users = users.merge(tmp, left_index=True, right_index=True)

In [10]:
del df, tmp
gc.collect()

43

In [11]:
users.head()

,user,item,pos
1801368,11098645,503292,60
1801368,11098645,675295,61
1801368,11098645,702285,62
1801368,11098645,334964,63
1801368,11098645,1312535,64


In [12]:
users.dtypes

user    int32
item    int32
pos     int32
dtype: object

In [13]:
tmp = df2.labels.str.split(' ').explode().astype('float32').rename('wgt').to_frame()
tmp['pos'] = np.tile( np.arange(100),tmp.shape[0]//100 ).astype('int32')
tmp.head()

,wgt,pos
1801251,-1.000000,0
1801251,36.650002,1
1801251,35.150002,2
1801251,29.150000,3
1801251,28.950001,4


In [14]:
users2 = df2[['user']].copy()#.reset_index()
users2.head()

,user
1801251,11098528
1801252,11098529
1801253,11098530
1801254,11098531
1801255,11098532


In [15]:
del df2
gc.collect()

22

In [16]:
users2 = users2.merge(tmp, left_index=True, right_index=True)

In [17]:
users2.head()

,user,wgt,pos
1801304,11098581,3.000,44
1801304,11098581,3.000,45
1801304,11098581,2.950,46
1801304,11098581,2.700,47
1801304,11098581,2.625,48


In [18]:
users = users.merge(users2,on=['user','pos'],how='left')

In [19]:
users.head()

,user,item,pos,wgt
0,11098602,147733,88,3.0
1,11098602,58690,89,3.0
2,11098602,42030,90,3.0
3,11098602,1678813,91,3.0
4,11098602,1362839,92,3.0


In [20]:
del users2, tmp
gc.collect()

79

In [21]:
users.shape

(180125100, 4)

# Add Targets

In [22]:
test_labels = cudf.read_parquet('../../data/train_data/test_labels.parquet')
test_labels = test_labels.loc[test_labels['type']==USE]
print( test_labels.shape )
test_labels.head()

(150817, 3)


,session,type,ground_truth
2,11098528,orders,"[990658, 950341, 1462506, 1561739, 907564, 369..."
4,11098530,orders,[409236]
5,11098531,orders,[1365569]
9,11098533,orders,"[935297, 652916, 334581, 1233050, 1366413, 118..."
12,11098534,orders,[223062]


In [23]:
tmp = test_labels.ground_truth.explode().astype('int32').rename('item')
tmp.head()

2     990658
2     950341
2    1462506
2    1561739
2     907564
Name: item, dtype: int32

In [24]:
test_users = test_labels[['session']].astype('int32').copy()
test_users.columns = ['user']

In [25]:
test_users = test_users.merge(tmp,left_index=True,right_index=True)

In [26]:
del test_labels, tmp
gc.collect()

50

In [27]:
test_users[USE] = np.int8(1)

In [28]:
test_users.dtypes

user      int32
item      int32
orders    int32
dtype: object

In [29]:
test_users.head()

,user,item,orders
151131,11218884,1060768,1
151131,11218884,75232,1
151131,11218884,73547,1
151131,11218884,1805003,1
151131,11218884,394861,1


In [30]:
users = users.merge(test_users, on=['user','item'], how='left').fillna(0)

In [31]:
del test_users
gc.collect()

40

In [32]:
users.head()

,user,item,pos,wgt,orders
0,11098618,968679,40,3.400000,0
1,11098618,845051,41,3.200000,0
2,11098618,1413290,42,3.175000,0
3,11098618,1082657,43,3.166667,0
4,11098618,1053428,44,3.025000,0


In [33]:
print(users.shape, users.dtypes)

(180125100, 5) user        int32
item        int32
pos         int32
wgt       float32
orders      int32
dtype: object


In [34]:
users = users.to_pandas()

# Filter Rows

In [35]:
%%time
tmp = users.groupby('user')[USE].agg('sum')
print( len(tmp) )
KEEP = tmp.loc[tmp>0].index.values
print( len(KEEP), len(KEEP)/len(tmp) )
del tmp
gc.collect()

1801251
134656 0.07475693282057859
CPU times: user 3.52 s, sys: 664 ms, total: 4.18 s
Wall time: 4.18 s


29

In [36]:
users = users.loc[users.user.isin(KEEP)].reset_index(drop=True)
print( users.shape )
gc.collect()

(13465600, 5)


0

In [37]:
users[USE].mean(), 1/users[USE].mean()

(0.016749940589353612, 59.70170429354284)

# Save

In [39]:
users.dtypes

user        int32
item        int32
pos         int32
wgt       float32
orders      int32
dtype: object

In [41]:
users[USE] = users[USE].astype('int8')

In [42]:
users.to_parquet(f'../../data/train_with_features/train_v{VER}.pqt',index=False)